In [4]:
#!pip install gspread oauth2client mysql-connector-python

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import mysql.connector
from datetime import datetime, date
import time
import smtplib
from email.message import EmailMessage
import ssl
from threading import Thread

# ======================================
#       CONFIGURAÇÕES (OBRIGATÓRIO)
# ======================================
CONFIG = {
    "mysql": {
        "host": "SEU IP",
        "user": "SEU ACESSO",
        "password": "SUA SENHA",
        "database": "SEU BANCO DE DADOS"
    },
    "email": {
        "endereco": "SEU EMAIL",
        "senha": "SUA SENHA DE 16 DIGITOS", ##Como ter acesso: https://myaccount.google.com/apppasswords
        "smtp_server": "smtp.gmail.com",
        "smtp_port": 465
    },
    "sheets": {
        "spreadsheet_id": "ID DO LINK DO SEU GOOGLE SHEETS",
        "sheet_name": "PAGINA QUE QUER CAPTURAR",
        "credentials_json": "ARQUIVO DE CREDENTIAL" ## video de como conseguir a credentials https://www.youtube.com/watch?v=1QFLfvvjmkQ&pp=ygUnY29tbyBmYXplciBjcmVkZW50aWFscyBubyBnb29nbGUgc2hlZXRz
    },
    "check_interval": 1  # Verificação a cada 1 segundo
}

# ======================================
#         FUNÇÕES PRINCIPAIS
# ======================================

def formatar_valor(valor):
    return valor.strftime('%d/%m/%Y %H:%M:%S') if isinstance(valor, (datetime, date)) else valor

def setup_cabecalho(sheet):
    """Cria os cabeçalhos na planilha se necessário"""
    cabecalho = ["ID", "Telefone", "Horário", "Campanha"]
    existing_header = sheet.row_values(1)
    
    if existing_header != cabecalho:
        sheet.insert_row(cabecalho, index=1)
        print("✅ Cabeçalhos configurados")

def enviar_notificacao(lead):
    try:
        msg = EmailMessage()
        msg['Subject'] = f"🚨 NOVO LEAD: {lead['campanha']}"
        msg['From'] = CONFIG["email"]["endereco"]
        msg['To'] = CONFIG["email"]["endereco"]
        
        corpo = f"""
        ⚡ **Novo lead cadastrado** ⚡
        📞 {lead['telefone']}
        🕒 {lead['data']}
        📋 {lead['campanha']}
        🔗 https://docs.google.com/spreadsheets/d/{CONFIG['sheets']['spreadsheet_id']}
        """
        msg.set_content(corpo.strip())
        
        with smtplib.SMTP_SSL(
            CONFIG["email"]["smtp_server"],
            CONFIG["email"]["smtp_port"],
            context=ssl.create_default_context()
        ) as server:
            server.login(CONFIG["email"]["endereco"], CONFIG["email"]["senha"])
            server.send_message(msg)
    except Exception as e:
        print(f"Erro e-mail: {e}")

def sincronizar_dados():
    while True:
        try:
            conn = mysql.connector.connect(**CONFIG["mysql"])
            cursor = conn.cursor(dictionary=True)
            
            creds = ServiceAccountCredentials.from_json_keyfile_name(
                CONFIG["sheets"]["credentials_json"],
                ["https://www.googleapis.com/auth/spreadsheets"]
            )
            client = gspread.authorize(creds)
            sheet = client.open_by_key(CONFIG["sheets"]["spreadsheet_id"]).worksheet(CONFIG["sheets"]["sheet_name"])
            
            # Configurar cabeçalhos
            setup_cabecalho(sheet)
            
            ultimo_id = -1
            print("🔁 Sistema em operação contínua (Ctrl+C para parar)")
            
            while True:
                try:
                    # Buscar último ID considerando a linha de cabeçalho
                    col_ids = sheet.col_values(1)[1:]  # Ignora o cabeçalho
                    current_last_id = int(col_ids[-1]) if col_ids else -1
                    ultimo_id = max(ultimo_id, current_last_id)
                    
                    cursor.execute(f"""
                        SELECT 
                            id,
                            telefone,
                            data,
                            campanha 
                        FROM [Sua tabela] 
                        WHERE id > {ultimo_id}
                        ORDER BY id ASC
                        LIMIT 1
                    """)
                    novo_lead = cursor.fetchone()
                    
                    if novo_lead:
                        # Mapear dados na ordem correta
                        dados = [
                            novo_lead['id'],
                            novo_lead['telefone'],
                            formatar_valor(novo_lead['data']),
                            novo_lead['campanha']
                        ]
                        
                        sheet.append_row(dados)
                        print(f"✅ Novo lead ID: {novo_lead['id']}")
                        Thread(target=enviar_notificacao, args=(novo_lead,)).start()
                        ultimo_id = novo_lead['id']
                    
                    time.sleep(CONFIG["check_interval"])
                
                except mysql.connector.Error as err:
                    print(f"Reconectando ao MySQL... Erro: {err}")
                    time.sleep(2)
                    conn.reconnect()
        
        except Exception as e:
            print(f"⚡ Reiniciando sistema. Erro: {e}")
            time.sleep(5)

# ======================================
#           EXECUÇÃO PERMANENTE
# ======================================
if __name__ == "__main__":
    sincronizar_dados()

⚡ Reiniciando sistema. Erro: 2005 (HY000): Unknown MySQL server host 'SEU IP' (11001)


KeyboardInterrupt: 